In [4]:
import sys
import pathlib as pl
import warnings

import pandas as pd
import numpy as np
import gtfs_kit as gk
import folium as fl

sys.path.append('../')

import make_gtfs as mg


DATA_DIR = pl.Path('../data')

%load_ext autoreload
%autoreload 2

warnings.filterwarnings(action='ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
path = DATA_DIR / 'auckland_light'
pfeed = mg.read_protofeed(path)
pfeed

ProtoFeed(meta=          agency_name           agency_url   agency_timezone start_date  \
0  Auckland Transport  https://at.govt.nz/  Pacific/Auckland   20140101   

   end_date  speed_route_type_3  speed_route_type_7  
0  20150101                  20                  30  , service_windows=  service_window_id start_time  end_time  monday  tuesday  wednesday  \
0    weekday_peak_1   07:00:00  09:00:00       1        1          1   

   thursday  friday  saturday  sunday  
0         1       1         0       0  , shapes=  shape_id                                           geometry
0     shp1  LINESTRING (174.76703 -36.84444, 174.76740 -36...
1     shp2  LINESTRING (174.74414 -36.84742, 174.74380 -36..., frequencies=  route_short_name route_long_name  route_type shape_id service_window_id  \
0                A    A nice route           3     shp1    weekday_peak_1   

   frequency  direction  speed  
0          1          1   22.0  , stops=None, speed_zones=                               

In [19]:
pfeed.frequencies

,route_short_name,route_long_name,route_type,shape_id,service_window_id,frequency,direction,speed
0,A,A nice route,3,shp1,weekday_peak_1,1,1,22.0


In [12]:
sz = pfeed.speed_zones
display(sz)

m = fl.Map(tiles="CartoDB Positron")
fl.GeoJson(
#     sz[lambda x: x.route_type == 3],
    sz,
    tooltip=fl.GeoJsonTooltip(["speed_zone_id", "speed"])
).add_to(m)

bounds = sz.total_bounds
bounds = [(bounds[1], bounds[0]), (bounds[3], bounds[2])]  # rearrange for Folium
m.fit_bounds(bounds)
m

,geometry,route_type,speed_zone_id,speed
0,"POLYGON ((174.78289 -36.87755, 174.74380 -36.8...",NaN,default,inf


In [16]:
pfeed.speed_zones.route_type.iat[0] = 3

In [18]:
feed = mg.build_feed(pfeed, stop_offset=0)

In [ ]:
f = (
    feed.routes[['route_id', 'route_short_name']]
    .merge(pfeed.frequencies)
    .merge(pfeed.service_windows)
    .sort_values('route_id')
)
f

In [ ]:
# Map some trips
prefix = 't-rB-weekday_peak_1-07:00:00'
t = (
    feed.trips
    .loc[lambda x: x.trip_id.str.startswith(prefix)]
)
tids = [t.trip_id.iat[0], t.trip_id.iat[-1]]
feed.map_trips(tids, include_arrows=True, include_stops=True)
    